In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, DateType
from pyspark.sql.functions import trim, col

In [0]:
# read from the bronze table 
df = spark.table("workspace.bronze.erp_loc_a101")

In [0]:
%sql 
Describe workspace.bronze.erp_loc_a101

In [0]:
# Contry Normalization 
df = df.withColumn(
    "CNTRY",
    F.when(col("CNTRY") == "DE", "Germany")
     .when(col("CNTRY").isin("US", "USA"), "United States")
     .when((col("CNTRY") == "") | col("CNTRY").isNull(), "n/a")
     .otherwise(col("CNTRY"))
)
df.show(10)

In [0]:
# Rename columns 
RENAME_MAP = {
    "cid": "customer_number",
    "cntry": "country"
}

for old_name, new_name in RENAME_MAP.items():
    df = df.withColumnRenamed(old_name, new_name) 
df.show(10)


In [0]:
# sanity check 
df.display()

In [0]:
# write to the silver table  
df.write.mode("overwrite").format("delta").saveAsTable("workspace.silver.erp_customer_location")

In [0]:

%sql
SELECT * FROM workspace.silver.erp_customer_location LIMIT 10